# Custom Training StyleGan3

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [22]:
print('Installing PyTorch...')
#! pip install torch 1>/dev/null
! pip3 install torch torchvision torchtext
print('Installing torchvision...')
#! pip install torchvision 1>/dev/null
print('Installing numpy...')
! pip install numpy 1>/dev/null
print('Installing ninja...')
! pip install ninja
print('Installing matplotlib...')
! pip install matplotlib 1>/dev/null
print('All required packages have been successfully installed!')

Installing PyTorch...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Installing torchvision...
Installing numpy...
Installing ninja...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Installing matplotlib...
All required packages have been successfully installed!


## Install Repo to Cloud Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [3]:
import os
if os.path.isdir("notebooks/notebook-sg3"):
    %cd "notebooks/notebook-sg3/stylegan3"
else:
    #install script
    !mkdir notebook-sg3
    %cd notebook-sg3
    !git clone https://github.com/NVlabs/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets

/notebooks/notebook-sg3
Cloning into 'stylegan3'...
remote: Enumerating objects: 207, done.
remote: Total 207 (delta 0), reused 0 (delta 0), pack-reused 207
Receiving objects: 100% (207/207), 4.16 MiB | 1.31 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Updating files: 100% (81/81), done.
/notebooks/notebook-sg3/stylegan3


In [8]:
import os, time
import torch
import pickle
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import numpy as np
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.dpi'] = 200  # I would like to make it higher, but too slow
import matplotlib.pyplot as plt
import json
from PIL import Image, ImageFont, ImageDraw

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
%cd "/notebooks/notebook-sg3/stylegan3"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

/notebooks/notebook-sg3/stylegan3


## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [11]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/notebooks/porkemons256.zip"

!unzip {zip_path} -d /notebooks/

Archive:  /notebooks/porkemons256.zip
   creating: /notebooks/s/
  inflating: /notebooks/s/staraptor.jpg  
  inflating: /notebooks/__MACOSX/s/._staraptor.jpg  
  inflating: /notebooks/s/taillow.jpg  
  inflating: /notebooks/__MACOSX/s/._taillow.jpg  
  inflating: /notebooks/s/charizard.jpg  
  inflating: /notebooks/__MACOSX/s/._charizard.jpg  
  inflating: /notebooks/s/infernape.jpg  
  inflating: /notebooks/__MACOSX/s/._infernape.jpg  
  inflating: /notebooks/s/pidove.jpg  
  inflating: /notebooks/__MACOSX/s/._pidove.jpg  
  inflating: /notebooks/s/bulbasaur.jpg  
  inflating: /notebooks/__MACOSX/s/._bulbasaur.jpg  
  inflating: /notebooks/s/banette.jpg  
  inflating: /notebooks/__MACOSX/s/._banette.jpg  
  inflating: /notebooks/s/articuno.jpg  
  inflating: /notebooks/__MACOSX/s/._articuno.jpg  
  inflating: /notebooks/s/pyroar.jpg  
  inflating: /notebooks/__MACOSX/s/._pyroar.jpg  
  inflating: /notebooks/s/magmortar.jpg  
  inflating: /notebooks/__MACOSX/s/._magmortar.jpg  
  infla

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
!python dataset_tool.py --help

In [12]:
#you don't need to edit anything here
!python dataset_tool.py --source=/notebooks/s --dest=/notebooks/notebook-sg3/stylegan3/datasets/pokemons256.zip
#!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

100%|████████████████████████████████████████| 721/721 [00:05<00:00, 135.20it/s]


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [16]:
!python train.py --help for help

^C
Traceback (most recent call last):
  File "train.py", line 17, in <module>
    import torch
  File "/opt/conda/lib/python3.8/site-packages/torch/__init__.py", line 28, in <module>
    from .torch_version import __version__ as __version__
  File "/opt/conda/lib/python3.8/site-packages/torch/torch_version.py", line 3, in <module>
    from pkg_resources import packaging  # type: ignore[attr-defined]
  File "/opt/conda/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3267, in <module>
    def _initialize_master_working_set():
  File "/opt/conda/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3241, in _call_aside
    f(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3279, in _initialize_master_working_set
    working_set = WorkingSet._build_master()
  File "/opt/conda/lib/python3.8/site-packages/pkg_resources/__init__.py", line 564, in _build_master
    ws = cls()
  File "/opt/conda/lib/python3.8/site-packages/pkg_

In [21]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "pokemons.zip"
#how often should the model generate samples and a .pkl file
snapshot_count = 50
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None

#resume

!python train.py --outdir=/notebooks/notebook-sg3/stylegan3/results --cfg=stylegan3-r --data=/notebooks/notebook-sg3/stylegan3/datasets/pokemons.zip \
    --gpus=1 --batch=32 --batch-gpu=4 --gamma=8 --mirror=1 --kimg=100 --snap=1 \
    --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl

#don't edit this unless you know what you're doing :)
#!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --augpipe="bg"
#stylegan3-r-ffhqu-256x256.pkl --kimg=5000 --cbase=16384 --cfg=stylegan3-r --gpus=1 --batch=32 --gamma=2 --batch-gpu=8 --snap=10


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 65536,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9988915792636801,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 8.0,
    "blur_init_si

# Generate Images

##Random Seeds

In [ ]:
pip install opensimplex

In [ ]:
snapshot="/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00020-sex-11gb-gpu-bg-resumecustom/network-snapshot-000008.pkl"

!python generate.py generate-images --network={snapshot} --seeds=1-1000 --trunc=1.3

##Growing

In [ ]:
import glob
model_names = glob.glob("results/00001-a-mirror-mirrory-11gb-gpu-bg/*.pkl") 

In [ ]:
for model in model_names:
  !python generate.py generate-images \
  --network /content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/{model}\
   --seeds=69 --trunc 0.8 --outdir ./results/growing_a/{model}

##Latent Walk

In [ ]:
snapshot="/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00020-sex-11gb-gpu-bg-resumecustom/network-snapshot-000008.pkl"
seednumbers = "964,977,911,936,833,744,774,790,467,964"
framenumber = "1000"

!python generate.py generate-latent-walk --network={snapshot} --seeds={seednumbers} --frames {framenumber} --trunc=1.3
# !python generate.py generate-latent-walk --network=/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00006-W-11gb-gpu-bg-resumecustom/network-snapshot-000028.pkl --seeds=15,21,31,38,58,69,217,234,279,288,15 --frames 300 --trunc=0.7